# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [5]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,✅ Trip Verified | Credit to BA - Flew to Sin...
1,✅ Trip Verified | The check in area for prem...
2,✅ Trip Verified | The flight took off a littl...
3,✅ Trip Verified | BA is great in the air! But....
4,✅ Trip Verified | I had broken my leg 2 days ...


In [7]:
df.to_csv("data/BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [8]:
df.dropna()

,reviews
0,✅ Trip Verified | Credit to BA - Flew to Sin...
1,✅ Trip Verified | The check in area for prem...
2,✅ Trip Verified | The flight took off a littl...
3,✅ Trip Verified | BA is great in the air! But....
4,✅ Trip Verified | I had broken my leg 2 days ...
...,...
995,✅ Trip Verified | BA836 on 19 Jan. Excellent ...
996,✅ Trip Verified | Whatever happened to the Wo...
997,✅ Trip Verified | Kuala Lumpur to Miami via L...
998,✅ Trip Verified | Zurich to Heathrow. Last ye...


In [14]:
df[["Status", "Comments"]] = df["reviews"].str.split("|", expand=True)
df[["Status", "Comments"]]

,Status,Comments
0,✅ Trip Verified,Credit to BA - Flew to Singapore recently a...
1,✅ Trip Verified,The check in area for premium classes at Ga...
2,✅ Trip Verified,The flight took off a little late due to Hea...
3,✅ Trip Verified,BA is great in the air! But... they sell you ...
4,✅ Trip Verified,I had broken my leg 2 days before our return...
...,...,...
995,✅ Trip Verified,BA836 on 19 Jan. Excellent short flight from...
996,✅ Trip Verified,Whatever happened to the World's Favourite A...
997,✅ Trip Verified,Kuala Lumpur to Miami via London in business...
998,✅ Trip Verified,Zurich to Heathrow. Last year they started b...
